## Variational Autoencoders Testing
Trying out the feasibility of variational autoencoders for music recommender systems.

In [125]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

import sklearn
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import *

### Data Input

Example entry in a playlist file:

```
"info": {
        "generated_on": "2017-12-03 08:41:42.057563", 
        "slice": "0-999", 
        "version": "v1"
    }, 
"playlists": [
    {
        "name": "Throwbacks", 
        "collaborative": "false", 
        "pid": 0, 
        "modified_at": 1493424000, 
        "num_tracks": 52, 
        "num_albums": 47, 
        "num_followers": 1, 
        "tracks": [
            {
                "pos": 0, 
                "artist_name": "Missy Elliott", 
                "track_uri": "spotify:track:0UaMYEvWZi0ZqiDOoHU3YI", 
                "artist_uri": "spotify:artist:2wIVse2owClT7go1WT98tk", 
                "track_name": "Lose Control (feat. Ciara & Fat Man Scoop)", 
                "album_uri": "spotify:album:6vV5UrXcfyQD1wu4Qo2I9K", 
                "duration_ms": 226863, 
                "album_name": "The Cookbook"
            }, 
            ...
         ],
        "num_edits": 6, 
        "duration_ms": 11532414, 
        "num_artists": 37
     }, 
```

In [126]:
data_path = 'large_files/'

In [127]:
f_start = 0
f_end = 999
num_files = 10
counter = 0

track_codes = {}
track_counts = {}
playlists = []

for i in range(num_files) : 
    with open(data_path + '/mpd.slice.{}-{}.json'.format(f_start, f_end)) as f : 
        data = json.load(f)
        
    for playlist in data['playlists'] : 
        playlist_dict = playlist.copy()
        playlist_dict.pop('tracks', None)
        
        for song in playlist['tracks'] : 
            track_name  = song['track_name']
            track_uri = song['track_uri']
            
            if track_name not in track_counts:
                track_counts[track_name] = 0
                
            if track_name not in track_codes:
                track_codes[track_name] = counter
                counter += 1
                
            track_counts[track_name] += 1
        
        last_song = playlist['tracks'][-1]['track_name']
        playlist_dict['last_song'] = track_codes[last_song]
#         playlist_dict['last_song'] = last_song
        
        playlists.append(playlist_dict)
            
    print ("done loading file", i)             
    f_start += 1000
    f_end += 1000

done loading file 0
done loading file 1
done loading file 2
done loading file 3
done loading file 4
done loading file 5
done loading file 6
done loading file 7
done loading file 8
done loading file 9


In [152]:
playlist_df = pd.DataFrame(playlists)
playlist_df = playlist_df.drop(['description', 'collaborative'], axis=1)

enc = sklearn.preprocessing.LabelBinarizer()
last_songs = enc.fit_transform(playlist_df['last_song'])

train_df, test_df, y_train, y_test = train_test_split(playlist_df, last_songs, test_size=0.2, random_state=836)
X_train = train_df.drop(['name', 'pid', 'last_song'], axis=1)
X_test = test_df.drop(['name', 'pid', 'last_song'], axis=1)

### KMeans Clustering

In [153]:
km = KMeans(n_clusters=10)
clusters =km.fit_predict(X_train)
X_train['cluster'] = clusters
X_test['cluster'] = km.predict(X_test)

display(X_train.head())

,duration_ms,modified_at,num_albums,num_artists,num_edits,num_followers,num_tracks,cluster
9753,9898463,1429488000,39,35,4,1,41,1
6528,6846312,1459555200,28,19,12,1,30,3
4290,11655830,1507680000,44,31,45,1,50,4
22,9077152,1509408000,39,37,15,2,42,4
3878,13706751,1473033600,33,11,5,5,67,3


### Word Embedding (For names)

In [ ]:
names = {}
count = 0
for name in train_df['name']:
    if name not in names:
        names[name] = count
        count += 1
print(len(names))

In [ ]:
embed_model = Sequential()
embed_model.add(Embedding(5000, 16, input_length=1))
embed_model.compile('rmsprop', 'mse')

In [ ]:
name_ids = train_df['name'].apply(lambda n: names[n])
embeddings = embed_model.predict(name_ids)
embeddings.shape

### Keras Model (Dense)

In [154]:
hidden_size = 50
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

In [155]:
model1 = Sequential()

model1.add(Dense(hidden_size, input_dim=input_dim, activation='relu'))
model1.add(Dense(hidden_size, activation='relu'))
model1.add(Dense(output_dim, activation='linear'))

model1.compile(optimizer='adam', loss='mean_absolute_error')

In [156]:
model1_history = model1.fit(X_train, y_train, batch_size=32, 
                            epochs=10, verbose=1, 
                            shuffle = True, validation_split=0.2)

Train on 6400 samples, validate on 1600 samples
Epoch 1/10
6400/6400 [==============================] - 4s 629us/step - loss: 444891.9355 - val_loss: 78116.3559
Epoch 2/10
6400/6400 [==============================] - 3s 515us/step - loss: 62679.8393 - val_loss: 50266.9962
Epoch 3/10
6400/6400 [==============================] - 3s 529us/step - loss: 41786.8608 - val_loss: 30751.4893
Epoch 4/10
6400/6400 [==============================] - 3s 526us/step - loss: 27214.7743 - val_loss: 20607.8004
Epoch 5/10
6400/6400 [==============================] - 3s 523us/step - loss: 18745.3708 - val_loss: 16950.2534
Epoch 6/10
6400/6400 [==============================] - 3s 524us/step - loss: 14255.1074 - val_loss: 8973.4102
Epoch 7/10
6400/6400 [==============================] - 3s 532us/step - loss: 17849.8867 - val_loss: 15221.9489
Epoch 8/10
6400/6400 [==============================] - 3s 525us/step - loss: 10465.1087 - val_loss: 0.0017
Epoch 9/10
6400/6400 [==============================] - 3s 5

In [157]:
model1.evaluate(X_test, y_test)

2000/2000 [==============================] - 0s 152us/step


0.00025214326265268029

### RNN/LSTM attempt

In [ ]:
model = Sequential()
model.add(Embedding(output_dim, hidden_size, input_length=input_len))
model.add(LSTM(hidden_size, return_sequences=True))
#model.add(LSTM(hidden_size, return_sequences=True))
#model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_dim, activation='softmax')))